### dev_notebook_

- Your development environment...

In [1]:
import pandas as pd
import requests
import json
import duckdb
import re
from shapely.geometry import Point
import geopandas as gpd
import numpy as np
from fuzzywuzzy import process
from tqdm import tqdm



# import numpy as np

In [2]:
def to_mercator(lat, long):
    # transform latitude/longitude data in degrees to pseudo-mercator coordinates in metres
    c = gpd.GeoSeries([Point(lat, long)], crs=4326)
    c = c.to_crs(3857)
    return c

def distance_meters(lat_start, long_start, lat_finish, long_finish):
    # return the distance in metres between to latitude/longitude pair point in degrees (i.e.: 40.392436 / -3.6994487)
    start = to_mercator(lat_start, long_start)
    finish = to_mercator(lat_finish, long_finish)
    return start.distance(finish)

In [3]:
# URL / ENDPOINT
base_url = 'https://datos.madrid.es/egob/'
resource = 'catalogo/300356-0-monumentos-ciudad-madrid.json'

endpoint = base_url + resource
endpoint

'https://datos.madrid.es/egob/catalogo/300356-0-monumentos-ciudad-madrid.json'

In [4]:
# MONUMENTOS MADRID - API Request
monuments = requests.get(endpoint).json()

In [5]:
dict_monuments = {'monument' : [monuments['@graph'][t]['title'] for t in range(len(monuments['@graph']))],
                  'address_mon' : [monuments['@graph'][a]['address']['street-address'] for a in range(len(monuments['@graph']))]}

dict_monuments['lat1'] = []
dict_monuments['lon1'] = []

for i in range(len(monuments['@graph'])):
    try:
        dict_monuments['lat1'].append(monuments['@graph'][i]['location']['latitude'])
        dict_monuments['lon1'].append(monuments['@graph'][i]['location']['longitude'])
    except:
        dict_monuments['lat1'].append(None)
        dict_monuments['lon1'].append(None)
        

#Failed list comprehensions due to lack of 'location' key in some items:
# lat = [bicimad['@graph'][l]['location']['latitude'] for l in range(len(bicimad['@graph']))]
# lon = [bicimad['@graph'][k]['location']['longitude'] for k in range(len(bicimad['@graph']))]

In [6]:
df_monuments = pd.DataFrame(dict_monuments)
df_monuments = df_monuments[df_monuments['lat1'].notna()]
df_monuments = df_monuments[df_monuments['monument'] == 'A los Abuelos']

In [7]:
choices = dict_monuments['monument']

In [8]:
process.extractOne("abuelos", choices)[0]

'A los Abuelos'

In [9]:
# Connexion duckdb

con = duckdb.connect(database='../data/bicimad_jup.db', read_only=False)
df_bicimad = con.execute('SELECT id, name, address, dock_bikes, "geometry.coordinates" FROM bicimad_stations').fetch_df()

In [10]:
df_bicimad = df_bicimad[df_bicimad['dock_bikes'] > 0].reset_index()

In [11]:
regex_loc = '[^, \[\]]+'

In [12]:
bici_loc = [(float(re.findall(regex_loc, df_bicimad['geometry.coordinates'][i])[1]),
             float(re.findall(regex_loc, df_bicimad['geometry.coordinates'][i])[0])) for i in range(len(df_bicimad))]

df_bicimad = df_bicimad.join(pd.DataFrame(bici_loc, columns=['lat2', 'lon2']))
df_bicimad.drop(columns='index', inplace=True)

In [13]:
full_df = df_monuments.merge(df_bicimad, how='cross')

In [14]:
%%time
full_df['pit'] = full_df.apply(lambda x : (x['lat1'] - x['lat2'])**2 + (x['lon1'] - x['lon2'])**2 , axis=1)

CPU times: total: 0 ns
Wall time: 5.56 ms


In [15]:
bici_min = pd.DataFrame(full_df.groupby(['monument', 'address_mon','lat1','lon1'])['pit'].min())

In [16]:
bici_min = bici_min.reset_index()
mon_bici = bici_min.merge(full_df, how='left').sort_values(by=['monument'])

In [17]:
%%time
mon_bici['Distance (m)'] = mon_bici.apply(lambda x : distance_meters(x['lat1'], x['lon1'], x['lat2'], x['lon2']), axis=1)

CPU times: total: 15.6 ms
Wall time: 24.7 ms


In [18]:
mon_bici['Walking time (min)'] = mon_bici.apply(lambda x : x['Distance (m)'] / 1000 * 12.5, axis=1)
mon_bici['Distance (m)'], mon_bici['Walking time (min)'] = mon_bici['Distance (m)'].astype(int), mon_bici['Walking time (min)'].astype(int)

In [34]:
# URL / ENDPOINT
base_url_b = 'https://openapi.emtmadrid.es/v2/mobilitylabs/user/login/'
resource_b = 'catalogo/300356-0-monumentos-ciudad-madrid.json'

# endpoint_b = base_url + resource
# endpoint_b

headers = {'email': 'magnofz@gmail.com', 'password' : 'Emt179357'}

In [35]:
test = requests.get(base_url_b, headers=headers).json()

In [30]:
test = requests.get(base_url_b).json()

In [36]:
test

{'code': '01',
 'description': 'Token 7a703eb6-e903-4df6-a74e-6709ff528fc5 extend  into control-cache Data recovered  OK, (lapsed: 45 millsecs)',
 'datetime': '2023-04-14T08:37:39.633185',
 'data': [{'XClientId': '09141f1a-bf02-4e23-80ee-e7fa7ea819d2',
   'nameApp': 'OPENAPI MobilityLabs',
   'updatedAt': '2023-04-13T17:30:10.4100000',
   'userName': 'magnofz86',
   'lastUpdate': {'$date': 1681454254274},
   'idUser': 'c87a8410-9cd9-4ac7-b993-1e725b289693',
   'priv': 'U',
   'tokenSecExpiration': 86399,
   'email': 'magnofz@gmail.com',
   'tokenDteExpiration': {'$date': 1681547853274},
   'flagAdvise': True,
   'accessToken': '7a703eb6-e903-4df6-a74e-6709ff528fc5',
   'apiCounter': {'current': 2,
    'dailyUse': 20000,
    'owner': 0,
    'licenceUse': 'Please mention EMT Madrid MobilityLabs as data source. Thank you and enjoy!',
    'aboutUses': 'If you need to extend the daily use of this API, please, register your App in Mobilitylabs and use your own X-ClientId and  passKey instead